In [18]:
import csv
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime

## Load the Dataset

In [13]:
bike_df = pd.read_csv("Bikeshare_Ridership_2017_Q3.csv")

In [14]:
bike_df.head()

,trip_id,trip_start_time,trip_stop_time,trip_duration_seconds,from_station_name,to_station_name,user_type
0,1253914,7/1/2017 0:00,7/1/2017 0:15,910,Princess St / Adelaide St E,424 Wellington St W,Member
1,1253915,7/1/2017 0:01,7/1/2017 0:15,837,Fort York Blvd / Capreol Crt,HTO Park (Queens Quay W),Casual
2,1253916,7/1/2017 0:01,7/1/2017 0:14,786,Fort York Blvd / Capreol Crt,HTO Park (Queens Quay W),Casual
3,1253917,7/1/2017 0:01,7/1/2017 0:25,1420,Elizabeth St / Edward St (Bus Terminal),Boston Ave / Queen St E,Casual
4,1253918,7/1/2017 0:01,7/1/2017 0:25,1437,Elizabeth St / Edward St (Bus Terminal),Boston Ave / Queen St E,Casual


In [19]:
bike_df['trip_start_time']=pd.to_datetime(bike_df['trip_start_time'])
bike_df['trip_stop_time']=pd.to_datetime(bike_df['trip_stop_time'])

In [31]:
bike_df['start_hour']=bike_df['trip_start_time'].apply(lambda x: x.hour)
bike_df['start_minute']=bike_df['trip_start_time'].apply(lambda x: x.minute)
bike_df['start_month']=bike_df['trip_start_time'].apply(lambda x: x.month)
bike_df['start_day']=bike_df['trip_start_time'].apply(lambda x: x.day)

bike_df['stop_hour']=bike_df['trip_stop_time'].apply(lambda x: x.hour)
bike_df['stop_minute']=bike_df['trip_stop_time'].apply(lambda x: x.minute)
bike_df['stop_month']=bike_df['trip_stop_time'].apply(lambda x: x.month)
bike_df['stop_day']=bike_df['trip_stop_time'].apply(lambda x: x.day)

In [32]:
bike_df.head()

,trip_id,trip_start_time,trip_stop_time,trip_duration_seconds,from_station_name,to_station_name,user_type,start_hour,start_minute,stop_hour,stop_minute,start_month,start_day,stop_month,stop_day
0,1253914,2017-07-01 00:00:00,2017-07-01 00:15:00,910,Princess St / Adelaide St E,424 Wellington St W,Member,0,0,0,15,7,1,7,1
1,1253915,2017-07-01 00:01:00,2017-07-01 00:15:00,837,Fort York Blvd / Capreol Crt,HTO Park (Queens Quay W),Casual,0,1,0,15,7,1,7,1
2,1253916,2017-07-01 00:01:00,2017-07-01 00:14:00,786,Fort York Blvd / Capreol Crt,HTO Park (Queens Quay W),Casual,0,1,0,14,7,1,7,1
3,1253917,2017-07-01 00:01:00,2017-07-01 00:25:00,1420,Elizabeth St / Edward St (Bus Terminal),Boston Ave / Queen St E,Casual,0,1,0,25,7,1,7,1
4,1253918,2017-07-01 00:01:00,2017-07-01 00:25:00,1437,Elizabeth St / Edward St (Bus Terminal),Boston Ave / Queen St E,Casual,0,1,0,25,7,1,7,1


## Calculate Arrival Rate

In [194]:
morning_df = bike_df[(bike_df['start_hour'] >= 7)&(bike_df['start_hour'] < 10)]
return_df = bike_df[(bike_df['stop_hour'] >= 7)&(bike_df['stop_hour'] < 10)]

In [139]:
morning_df['Value'] = np.nan

/anaconda2/envs/python3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [147]:
for i,row in morning_df.iterrows(): 
    if (row['start_minute'] >= 0) and (row['start_minute'] < 20): 
        morning_df.at[i, 'Value'] = 1
    elif (row['start_minute'] >= 20) and (row['start_minute'] < 40): 
        morning_df.at[i, 'Value'] = 2
    else:
        morning_df.at[i, 'Value'] = 3

In [189]:
def start_station(station_name):
    df = morning_df[morning_df['from_station_name'] == station_name]
    c = ['7:00-7:20', '7:20-7:40', '7:40-8:00', '8:00-8:20', '8:20-8:40', '8:40-9:00', 
                                    '9:00-9:20', '9:20-9:40', '9:40-10:00']
    count = pd.DataFrame(columns = c)
    gross = []
    for h in range(7,10,1):
        for i in range(1,4,1):
            counts = []
            for m in range(7,10,1):
                for d in range(1,32,1):
                    counts.append(len(df[(df['start_month'] == m) & (df['start_day'] == d) & 
                                        (df['start_hour']== h) & (df['Value'] == i)]))
            gross.append(counts)
    
    for j in range(0,9,1):
        count[c[j]] = gross[j]

    count = count.drop(count.index[len(count)-1])
    count.loc['Mean'] = count.mean()
    
    return count

In [191]:
Bay_St = start_station('Bay St / St. Joseph St');
Union = start_station('Union Station');
College_St = start_station('College St / Major St');
Queens = start_station('Queens Quay / Yonge St');
Madison = start_station('Madison Ave / Bloor St W');

In [193]:
with pd.ExcelWriter('BikeRentCount.xlsx') as writer:
    Bay_St.to_excel(writer, sheet_name = 'Bay_St')
    Union.to_excel(writer, sheet_name = 'Union')
    College_St.to_excel(writer, sheet_name = 'College_St')
    Queens.to_excel(writer, sheet_name = 'Queens')
    Madison.to_excel(writer, sheet_name = 'Madison')

## Calculate Return Rate

In [195]:
return_df['Value'] = np.nan
for i,row in return_df.iterrows(): 
    if (row['stop_minute'] >= 0) and (row['stop_minute'] < 20): 
        return_df.at[i, 'Value'] = 1
    elif (row['stop_minute'] >= 20) and (row['stop_minute'] < 40): 
        return_df.at[i, 'Value'] = 2
    else:
        return_df.at[i, 'Value'] = 3

/anaconda2/envs/python3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [196]:
def return_station(station_name):
    df = return_df[return_df['to_station_name'] == station_name]
    c = ['7:00-7:20', '7:20-7:40', '7:40-8:00', '8:00-8:20', '8:20-8:40', '8:40-9:00', 
                                    '9:00-9:20', '9:20-9:40', '9:40-10:00']
    count = pd.DataFrame(columns = c)
    gross = []
    for h in range(7,10,1):
        for i in range(1,4,1):
            counts = []
            for m in range(7,10,1):
                for d in range(1,32,1):
                    counts.append(len(df[(df['stop_month'] == m) & (df['stop_day'] == d) & 
                                        (df['stop_hour']== h) & (df['Value'] == i)]))
            gross.append(counts)
    
    for j in range(0,9,1):
        count[c[j]] = gross[j]

    count = count.drop(count.index[len(count)-1])
    count.loc['Mean'] = count.mean()
    
    return count

In [197]:
Bay_St_r = return_station('Bay St / St. Joseph St');
Union_r = return_station('Union Station');
College_St_r = return_station('College St / Major St');
Queens_r = return_station('Queens Quay / Yonge St');
Madison_r = return_station('Madison Ave / Bloor St W');

In [198]:
with pd.ExcelWriter('BikeReturnCount.xlsx') as writer:
    Bay_St_r.to_excel(writer, sheet_name = 'Bay_St')
    Union_r.to_excel(writer, sheet_name = 'Union')
    College_St_r.to_excel(writer, sheet_name = 'College_St')
    Queens_r.to_excel(writer, sheet_name = 'Queens')
    Madison_r.to_excel(writer, sheet_name = 'Madison')